In [1]:
## 导入需要的库
import numpy as np
import pandas as pd
import math
import os
import random
from matplotlib.pyplot import *
import matplotlib.pyplot as plt

from statsmodels.stats.weightstats import ztest

In [2]:
lb1 = [0, 9]
lb2 = [10]
lb3 = [1, 2, 3, 4, 5, 6, 7, 8]

In [5]:
## K-means聚类结果
from sklearn.cluster import KMeans
X = pd.read_csv("香港预处理后的数据归一化数据.csv")
X = X.drop(['Unnamed: 0'], axis = 1)
estimator = KMeans(n_clusters = 3)#构造聚类器
estimator.fit(X)#聚类
label2 = estimator.labels_ #获取聚类标签
label2

array([2, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1])

In [6]:
a1, a2, a3 = [], [], []
for j in range(0,len(X)):
    if label2[j] == 2:
        a1.append(j)
    elif label2[j] == 1:
        a2.append(j)
    else:
        a3.append(j)
print(a1)
print(a2)
print(a3)

[0]
[1, 3, 4, 5, 6, 7, 9, 10]
[2, 8]


In [7]:
## 三支聚类
data = X
data.head()

,0,1,2,3,4,5,6,7,8,9,...,235,236,237,238,239,240,241,242,243,244
0,0.736193,0.501542,0.816691,0.593069,0.647679,0.702279,0.672787,0.649062,0.631335,0.634301,...,0.612507,0.665645,0.680750,0.655391,0.711012,0.700231,0.651183,0.738339,0.622953,0.695193
1,0.342608,0.234420,0.412883,0.409494,0.437185,0.413305,0.502298,0.332576,0.356607,0.317715,...,0.298241,0.360469,0.402403,0.262128,0.383770,0.490490,0.334200,0.424446,0.450695,0.357240
2,0.457942,0.579726,0.473875,0.587997,0.539056,0.382939,0.695173,0.579591,0.539056,0.506647,...,0.550544,0.630311,0.378605,0.410466,0.609477,0.769004,0.424988,0.820954,0.303000,0.561870
3,0.269724,0.138360,0.331418,0.410485,0.538670,0.393172,0.460095,0.312516,0.425359,0.255574,...,0.242354,0.344429,0.344125,0.204187,0.324723,0.473141,0.238255,0.438652,0.239695,0.330825
4,0.362847,0.227501,0.417300,0.403319,0.429789,0.291033,0.518382,0.341012,0.362885,0.340715,...,0.275528,0.371694,0.296157,0.286968,0.371694,0.546898,0.288832,0.528461,0.237691,0.401852


In [8]:
### 设定正理想解和负理想解
n, m = len(data), len(data.T)

### 计算决策对象到正负理想解的距离
D1 = np.zeros((1,n));D0 = np.zeros((1,n)); Pr = []
for i in range(0,n):
    D1[0][i] = np.sqrt(np.sum(np.square(1 - data.iloc[i]))) 
    D0[0][i] = np.sqrt(np.sum(np.square(data.iloc[i])))
    Pr.append(D0[0][i]/(D0[0][i] + D1[0][i]))
print(Pr)

[0.6668528595995902, 0.3699945802864106, 0.5257753109902581, 0.327653184204264, 0.3726683236181743, 0.40109337060868466, 0.34949459662695304, 0.3891823249084575, 0.510276561045845, 0.38911305129933954, 0.3511928713146707]


In [9]:
def sunshi(x,singam):
    lamda = np.zeros((len(x),6));
    juhe = np.zeros((1,6));
    
    # 计算相对损失函数
    for i in range(0,len(x)):
        lamda[i,1] = singam[i] * x[i];
        lamda[i,2] = x[i];
        lamda[i,3] = 1-x[i];
        lamda[i,4] = singam[i] * (1-x[i]);
    
    # 计算聚合相对效用函数(均值聚合)
    juhe = np.mean(lamda,axis=0)
    
    return juhe

In [12]:
# 计算不同对象的聚合相对损失函数矩阵
singam = [0.3]*m
lamda = np.zeros((n,6));
for i in range(0,n):
    x = data.iloc[i]
    lamda[i] = sunshi(x ,singam)

#根据聚合相对损失函数计算不同对象的初始阈值
yuzhi = np.zeros((n,2))
for i in range(0,n):
    for j in range(0,len(lamda.T)):
        yuzhi[i][0] = (lamda[i][3]-lamda[i][4])/(lamda[i][3]-lamda[i][4]+lamda[i][1])
        yuzhi[i][1] = (lamda[i][4])/(lamda[i][4]+lamda[i][2]-lamda[i][1])
yuzhi = pd.DataFrame(yuzhi)
yuzhi.head()

,0,1
0,0.534453,0.174140
1,0.802808,0.427843
2,0.675256,0.276370
3,0.833820,0.479599
4,0.803855,0.429466


In [13]:
B1, B2, B3 = [], [], []
for i in range(n):
    if Pr[i] > yuzhi.loc[i][0]:
        B1.append(i)
    elif Pr[i]> yuzhi.loc[i][1] and Pr[i] < yuzhi.loc[i][0]:
        B2.append(i)
    else:
        B3.append(i)
print(B1)
print(B2)
print(B3)

[0]
[2, 5, 8]
[1, 3, 4, 6, 7, 9, 10]


In [14]:
## 基于遗憾理论的灰度关联聚类方法
r1 = np.zeros((n,m))
r2 = np.zeros((n,m))

for i in range(n):
    for j in range(m):
        r1[i][j] = 1/(1.5-data.loc[i][j])
        r2[i][j] = 1/(data.loc[i][j]+0.5)

In [15]:
# 建立感知效用矩阵
def Juzhen(alpha, sigma):
    U = np.zeros((n,m))
    R = r1/(r1+r2)
    G = 1 - np.exp(sigma * abs(r1-1))
    Q = 1 - np.exp(-sigma * abs(r1-1))
    for i in range(n):
        for j in range(m):
            U[i][j] = (R[i][j]) ** alpha + G[i][j] + Q[i][j] 
    return U

In [16]:
U = Juzhen(0.5,0.15)
U

array([[0.78403927, 0.70765177, 0.80655006, ..., 0.78466966, 0.74887506,
        0.77171987],
       [0.64866329, 0.6049876 , 0.67545994, ..., 0.6797588 , 0.6894049 ,
        0.65433953],
       [0.69204056, 0.73458493, 0.69779456, ..., 0.80766868, 0.63303094,
        0.72855487],
       ...,
       [0.7200755 , 0.67305351, 0.73048746, ..., 0.72156203, 0.68488782,
        0.72576726],
       [0.67993773, 0.67114605, 0.66758658, ..., 0.68478515, 0.61285587,
        0.68060069],
       [0.66659198, 0.62720412, 0.70522635, ..., 0.6639048 , 0.62272218,
        0.65806798]])

In [17]:
# 计算聚类系数
u = []
w = [0.1,0.2,0.5,0.05,0.15]
for i in range(n):
    u.append(np.sum(w * U[i][m-5:]))
print(u)

[0.7744553898359864, 0.6719495803596165, 0.7601118755828794, 0.6605380661187521, 0.687369621007067, 0.6683259637674522, 0.6540738098840091, 0.6797708986524938, 0.7211791696148822, 0.6754406208643672, 0.6573048880198805]


In [18]:
C1, C2 = [], []
for i in range(n):
    if u[i] > np.mean(u):
        C1.append(i)
    else:
        C2.append(i)
print(C1)
print(C2)

[0, 2, 8]
[1, 3, 4, 5, 6, 7, 9, 10]


In [19]:
from numpy import unique
from sklearn.cluster import AffinityPropagation
import numpy as np
from sklearn.preprocessing import StandardScaler

model = AffinityPropagation(damping=0.5, max_iter=200, convergence_iter=15, copy=True, preference=None,    
                            affinity='euclidean', verbose=False) # 设置damping : 阻尼系数，取值[0.5,1)
# 匹配模型
model.fit(X)
yhat = model.predict(X)  # yhat为集群结果
clusters = len(unique(yhat))  # 类别

print(yhat)

[0 1 2 1 1 1 1 1 3 1 1]


In [20]:
D1, D2, D3, D4 = [], [], [], []
for i in range(n):
    if yhat[i] == 0:
        D1.append(i)
    if yhat[i] == 1:
        D2.append(i)
    if yhat[i] == 2:
        D3.append(i)
    if yhat[i] == 3:
        D4.append(i)
print(D1)
print(D2)
print(D3)
print(D4)

[0]
[1, 3, 4, 5, 6, 7, 9, 10]
[2]
[8]
